In [4]:
import os
import sys
import cv2
import time
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

In [5]:
hands = {'rock': 0, 'paper': 1, 'scissors': 2, 'other': 3}

def oneHotEncode(labels, n_classes=4):
    return np.eye(n_classes, dtype=int)[labels]

# image data augmentation
def horizontalFlip(img):
    return cv2.flip(img, 1)

def verticalFlip(img):
    return cv2.flip(img, 0)

def grayScale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def rotate(img, angle):
    return cv2.rotate(img, angle)

In [6]:
def augment():
    for hand in hands.keys():
        for filename in os.listdir(hand):
            img = cv2.imread(os.path.join(hand, filename))
            filename = filename.split('.')[0]
            cv2.imwrite(filename=os.path.join(hand, "{}_hflip.jpg".format(filename)), img=horizontalFlip(img))
            cv2.imwrite(filename=os.path.join(hand, "{}_vflip.jpg".format(filename)), img=verticalFlip(img))
            cv2.imwrite(filename=os.path.join(hand, "{}_grayscale.jpg".format(filename)), img=grayScale(img))
            cv2.imwrite(filename=os.path.join(hand, "{}_rotate90cw.jpg".format(filename)), 
                img=rotate(img, cv2.ROTATE_90_CLOCKWISE))
            cv2.imwrite(filename=os.path.join(hand, "{}_rotate90ccw.jpg".format(filename)), 
                img=rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))


In [7]:
def preprocess():
    start = time.time()
    images = []
    labels = []
    for hand in hands.keys():
        for filename in os.listdir(hand):
            img = cv2.imread(os.path.join(hand, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (227, 227))
            img = np.reshape(img, (227,227,1))
            images.append(img)
            labels.append(hands[hand])
    labels = oneHotEncode(labels)
    end = time.time()
    print("time taken for preprocessing and labelling", end - start)
    return np.array(images), labels

In [9]:
images, labels = preprocess()

time taken for preprocessing and labelling 34.013641119003296


In [10]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(images, labels, test_size=0.05, shuffle=True)

In [11]:
# Test/Train dataset shapes
print(Xtest.shape)
print(Xtrain.shape)
print(Ytest.shape)
print(XYtrain.shape)

(500, 4)
(9495, 227, 227, 1)


In [13]:
# build model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(227,227,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [14]:
history = model.fit(x=Xtrain, y=Ytrain, epochs=3)

Train on 9495 samples
Epoch 1/3
9495/9495 [==============================] - 920s 97ms/sample - loss: 2.4881
Epoch 2/3
9495/9495 [==============================] - 831s 88ms/sample - loss: 0.6435
Epoch 3/3
9495/9495 [==============================] - 818s 86ms/sample - loss: 0.2619


In [15]:
model.evaluate(x=Xtest, y=Ytest)

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.10518057491002808

In [16]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

KeyError: 'accuracy'

In [17]:
model.save("rps_model3.h5")

In [18]:
from tensorflow.keras.models import load_model
import tensorflow as tf
print(tf.__version__)
new_model = load_model("rps_model3.h5")

2.0.0


In [19]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 225, 225, 64)      640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 110, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 128)      

In [40]:
imag = np.asarray([Xtest[44]])
imag = tf.cast(imag, tf.float32)

print(imag.shape)
print(new_model.predict(imag))
print(Ytest[44])
print(np.argmax(new_model.predict(imag)))


(1, 227, 227, 1)
[[6.5689551e-30 2.9463283e-28 4.3612834e-31 1.0000000e+00]]
[0 0 0 1]
3


In [24]:
x = Xtest[1]
y = Ytest[1]
# print(Xtest[1].shape)
model.predict(np.asarray([x]))

TypeError: Value passed to parameter 'input' has DataType uint8 not in list of allowed values: float16, bfloat16, float32, float64